In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("20230505_Messages.csv")
df.head()

,Datetime,Message
0,2023-03-01 14:28:13,Please leave a one line intro when you join an...
1,2023-03-01 14:29:24,I'm Pranjal Mehta. Cofounded ePlane.ai (electr...
2,2023-03-01 14:29:26,"Hey folks,"
3,2023-03-01 14:29:50,I'm Nirant K. ML/LLM Consultant out of BLR. Mo...
4,2023-03-01 14:30:27,Hey


In [7]:
# Group by week
df["Datetime"] = pd.to_datetime(df["Datetime"])
df['Week'] = df['Datetime'].dt.isocalendar().week
df['Date'] = df['Datetime'].dt.date

# Group by Date
daily_df = df.groupby('Date').agg({'Message': ' \n '.join}).reset_index()
daily_df = pd.DataFrame(daily_df)
len(daily_df)

# # Group by Week
# weekly_df = df.groupby('Week').agg({'Message': ' \n '.join}).reset_index()
# weekly_df = pd.DataFrame(messages_df)
# print(weekly)
# print(weekly_df["Message"][9])

63

In [8]:
daily_df["wc"] = daily_df["Message"].apply(lambda x: len(x.split()))

In [9]:
daily_df["wc"].describe()

count      63.000000
mean      900.492063
std       727.693703
min         8.000000
25%       286.500000
50%       695.000000
75%      1498.000000
max      2540.000000
Name: wc, dtype: float64

In [10]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0)

In [11]:
text_splitter = CharacterTextSplitter.from_tiktoken_encoder()

In [23]:
plain_text = daily_df["Message"][54]
texts = text_splitter.split_text(plain_text)

In [24]:
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
docs = [Document(page_content=t) for t in texts[:3]]

In [76]:
prompt_template = """This is a group chat. Write a detailed, exhaustive research bullet points for each section in Markdown format for titles:

{text}

Research with weblinks where relevant:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
chain = load_summarize_chain(ChatOpenAI(temperature=0), chain_type="map_reduce", return_intermediate_steps=True, map_prompt=PROMPT, combine_prompt=PROMPT)
output_text = chain({"input_documents": docs}, return_only_outputs=True)["output_text"]
print(output_text)

# OpenAI not needing data

- OpenAI claims they don't need more data, but this is highly doubtful as they would happily take more data if it was available.
- They have exhausted text data for GPT and need more data for fine-tuning layers like RLHF.
- OpenAI doesn't use API data, and chatting on their website probably won't be used as is (which they state in the free version).
- Ideally, if they really don't need data, they should give a zero knowledge guarantee.
- OpenAI doesn't use input or output, but they could use intermediate layer information. It's unclear if their team can eyeball it to improve the system.
- They do need more data, just not API platform customer data.
- OpenAI is actively seeking more data to train their models from the industry because they have exhausted internet and academic datasets.
- Over time, they may get to your request.

# Data Partnership

- OpenAI has data partners, but a list of them is not readily available.
- Their platform API data policy is publ

In [64]:
docs

[Document(page_content='OpenAI not needing data: Highly doubt that claim, they like everyone else would happily take more data. Esp when they are out of training data for GPT (they have exhausted text data). But also as feedback for fine-tuning layers like RLHF. The key there is they don\'t use API day, chatting on website probably will be used as is (which they say as well, in the free version). Ideally if they really don\'t need data, they should give a zero knowledge guarantee. They don\'t use input or output. But could they use intermediate layer info, can their folks eyeball it to improve the system. All those are probably open questions. \n I think multinationals too will move to start using them. Their models are quite good and it is just a matter of time. \n They do need more data, just not api platform customer data. \n Data Partnership: Interesting. \n Anyone have a list of data partners for Open AI? \n Nah... They don\'t talk about how they train their new models either in t